# PGE Creation
This notebook is the driver by which the *PGE_NAME_PLACEHOLDER_notebook* will be processed in order to be available as an SDS PGE. The notebook *PGE_NAME_PLACEHOLDER_submit_job* is then used to submit it to the SDS for execution.


### 1. Set-up
#### To start, we; enter the directory containing the notebook to be executed (this step assumes the repository has already been cloned locally), ensure that we are in the proper branch, then execute notebook-pge-wrapper to generate the files necessary to build and run the *papermill*'ed notebook as a PGE.

#### Make note of the branch being used (e.g. *main* in this example), it will become part of the job specification name and used when submitting the job in the next notebook.

In [ ]:
%%bash
# This is a temporary workaround
pip install --quiet -e /home/jovyan/notebook_pge_wrapper/

cd ~/PGE_NAME_PLACEHOLDER
git checkout main
notebook-pge-wrapper specs all

### 2. Commit changes
#### The build process retrieves source code for the PGE from the GitHub repository, so it is necessary to commit and push all changes before building/rebuilding the PGE.
#### If `git push` fails here, push the local repository to the remote manually (using JupyterLab or the Git CLI)

In [ ]:
%%bash
cd ~/PGE_NAME_PLACEHOLDER
git add -A
git commit -m"Results of notebook-pge-wrapper spec generation."
git push

### 3. Initialize the API client which will be used

When prompted for the HySDS host, include the protocol, e.g. https://my-mozart.jpl.nasa.gov
When prompted for "HySDS cluster authenticated", enter 'y' if the cluster requires a password to access.

In [ ]:
import os
from pathlib import Path
import otello
if not os.path.exists(f"{Path.home()}/.config/otello/config.yml"):
    otello.client.initialize()

### 4. Specify the repository and branch to the api
#### This creates a handle to the Jenkins build process for your repository/PGE.
#### *Set "account" and "repository_name" to the correct value for the remote repository you created for this PGE.*

In [ ]:
account = "YOUR GIT ORG/ACCOUNT NAME HERE"
repository_name = "PGE_NAME_PLACEHOLDER"
repository_url = f"https://github.com/{account}/{repository_name}.git"
branch = "main"

ci = otello.CI(repo=repository_url, branch=branch)

### 5. Registering the repository/job-type with the build pipeline
#### The job-type/PGE must be registered with the build process before it can be built.
#### Existing registrations for the job-type/PGE are removed first, to ensure that the cached versions on active worker nodes are invalidated and replaced.

In [ ]:
try:
    if ci.check_job_exists():
        print("Job exists, unregistering")
        ci.unregister()
except Exception as e:
    print("Exception : {}, trying to register anyway".format(e))

ci.register()

### 6. Build the PGE

In [ ]:
ci.submit_build()

### 7. Wait for the build to resolve, then check whether the build was successful or not

In [ ]:
import time
move_on = False
while not move_on:
    try:
        ci.get_build_status()
        move_on = True
    except Exception as e:
        print("Waiting for build to start")
        time.sleep(10)
    
build_stat = ci.get_build_status()
print("Build in progress.", end="")

while (build_stat['building']):
    time.sleep(10)
    print(".", end="", flush=True)
    build_stat = ci.get_build_status()

if (build_stat['result'] == 'FAILURE'):
    print("\nBuild failed.")
else:
    print("\nBuild completed successfully")


